# Fouille de texte avec Python

## [Notebook précédent](https://colab.research.google.com/drive/1HmtwxMbiCQz4wed-tY57Qf-AI5EpL6Ux?usp=sharing)

---

## Introduction

Dans cette étape, nous allons analyser les corpus textuels. La chaîne de traitement est la suivante:
- les quatre fichiers sont ouverts et chargés depuis Github
- le texte brut est structuré dans une liste (`list`) de dictionnaires (`dict`) Python: par détection de motifs, les différentes entrées sont séparées; les parties de chaque entrée sont elles aussi extraites
- à partir de ces dictionnaires, les données utiles pour construire nos graphiques sont extraites et on calcule des statistiques
- des graphiques sont créés pour analyser le corpus avec la librairie Python `Plotly`

Pour rappel, la question qu'on se pose est:

> Quelle est la représentation de cinq genres littéraires du XVIIIe siècle (poésie, théâtre, roman, littérature d'idées) dans un corpus datant du XIXe siècle de catalogues de vente de manuscrits ?

---

## Poser les bases

On commence par faire quelques petites choses utiles pour la suite du notebook.

### Uploader les fichiers dans notre drive

- Téléchargez les 4 fichiers disponibles [à ce lien](https://we.tl/t-uFDI3C3AMC), reçus par mail ou disponibles sur [le dépôt Git du cours](https://github.com/paulhectork/cours_ens2023_fouille_de_texte/tree/main/in) (dans ce cas, il faut cliquer sur un fichier et ensuite sur le bouton `Raw`, et télécharger le contenu qui s'affiche.
- Dézippez les si-besoin
- Uploadez les dans le notebook (via le petit bouton `Fichiers` sur la gauche de la fenête), sans les renommer, à la racine de ce dossier.

J'aurais bien aimé faire plus simple mais franchement j'ai beaucoup essayé sans y arriver :(

### Installer les dépendances

On installe les librairies utilisées lors de ce cours.

En Python, certaines fonctionnalités sont disponibles de base, mais la plupart sont organisées dans des libraires. Une librairie est un ensemble de fonctions rassemblées ensemble avec un but spécifique: visualisations, calcul... Certaines librairies sont disponibles de base en Python, d'autres sont développées par des tiers et doivent être installées. Une librairie peut aussi être appelée "module" ou "package".

In [ ]:
!pip install certifi==2022.12.7
!pip install charset-normalizer==3.0.1
!pip install idna==3.4
!pip install kaleido==0.2.1
!pip install pkg_resources==0.0.0
!pip install plotly==5.13.0
!pip install requests==2.28.2
!pip install tenacity==8.2.1
!pip install urllib3==1.26.14

### Importer les librairies utilisées

En Python, certaines fonctionnalités sont disponibles de base, mais la plupart sont organisées dans des *libraires*. Une librairie est un ensemble de fonctions rassemblées ensemble avec un but spécifique: visualisations, calcul... Certaines librairies sont disponibles de base en Python, d'autres sont développées par des tiers et doivent être installées. Une librairie peut aussi être appelée "module" ou "package".

In [ ]:
# syntaxe de base pour importer toute une librairie: `import <nom de librairie>`
import re  # librairie pour les expressions régulières (détection de motifs dans le texte)

# importer une partie d'une librairie
from statistics import median  # on importe la fonction `median` dans la librairie `statistics` pour calculer une valeur médiane
import plotly.graph_objects as go  # la librairie plotly est très lourde => on importe seulement une partie: `graph_objects`. on donne à `graph_objects` l'alias `go` avec `as`. Dans notre code, on appellera `go` pour utiliser le module `plotly.graph_objects`.

---

## Ouvrir les fichiers et lire leur contenu

(note: dans les notebooks jupyter, on charge les fichiers localement et on apprend à utiliser `os` par la même occasion)

Pour lire un fichier en Python, on utilise la syntaxe suivante:

```python
with open("chemin du fichier", mode) as fh:
  fh.read()
```

- avec `with... as`, on lance une fonction (ici, `open()` et on l'assigne à une variable (ici `fh`). Cette syntaxe permet de définir une variable pour un temps précis seulement.
- `open()` permet d'ouvrir un fichier. L'argument `mode` indique ce que l'on veut faire avec ce fichier. Ici, on utilise `mode="r"` pour lire le contenu du fichier. Pour écrire, c'est `mode="w"`.
- `fh.read()` permet de lire le contenu du fichier `fh`.

In [ ]:
def read_input_file(file_name):
    """
    `def` indique qu'on définit une fonction. 
    la syntaxe est: def nom_de_la_fonction(liste, de, nos, arguments).
    """
    with open( file_name, mode="r" ) as fh:  # on ouvre le fichier. le `mode= "r"` indique qu'on ouvre le fichier en lecture.
        corpus = fh.read()  # on lit le fichier
    # quand on "retourne" une variable avec `return`, on permet d'utiliser 
    # sa valeur en dehors de la fonction. les variables retournées sont le 
    # résultat de la fonction, toutes les autres sont supprimées quand la fonction se termine
    return corpus


corpus_idees = read_input_file("catalogue_idees.txt")
corpus_roman = read_input_file("catalogue_roman.txt")
corpus_poeme = read_input_file("catalogue_poeme.txt")
corpus_theatre = read_input_file("catalogue_theatre.txt")

# on imprime un de nos corpus. que remarquez-vous sur sa structure? 
print(corpus_idees)

---

## Structurer les corpus

Dans leur forme actuelle, il est difficile de faire sens de nos 4 corpus: ils ne sont pas structurés, et un ordinateur lit de manière bête et méchante: il lit lettre à lettre, ne fait pas la différence entre les mots, les lignes et encore moins les différentes entrées de catalogue!

### Identifier une structure: le programme des festivités

Pour pouvoir exploiter notre corpus, on va donc devoir le structurer. **N'importe quel travail de structuration, de lecture du texte commence par une vraie lecture, humaine du texte**: c'est comme ça que l'on reconnaît sa structure. On peut dire que la structure d'une entrée est la suivante:

```
Nom de l'auteur.ice
Écrit en [date]. Vendu en [date].
Dimensions: [nombre décimal].
[description en texte libre]
Prix: [nombre décimal] [monnaie] (en francs constants 1900: [nombre décimal].  # ligne optionnelle
[saut de ligne]
[saut de ligne]
```

Une fois qu'on a reconnu cette structure, la première étape est de la rendre explicite pour l'ordinateur: du texte brut, on doit passer à un format structuré où l'ordinateur comprenne la différence entre les entrées et entre les différentes parties d'une entrée.

**C'est cette étape le cœur du cours**: une fois que la structure d'un texte est compréhensible par un ordinateur, on peut faire quelque chose de ce texte avec des outils numériques: en cibler des parties, faire des filtres, quantifier... On doit permettre à un ordinateur de "lire" du texte.

On va chercher à représenter chaque entrée par **une liste (`list`) de dictionnaires (`dict`)**:
- **`list`**: une liste est une série de valeurs séparées par des virgules (`,`) qui s'écrit entre `[]`. Elle peut contenir n'importe quel type de données.
    - *exemple: `[ 1, "deux", ["liste", "imbriquée"]]`*
- **`dict`**: un dictionnaire permet d'associer deux éléments. 
    - Comme dans un vrai dictionnaire (où un mot est associé à sa définition), un premier élément (la clé) est associée à une deuxième (la valeur). 
    - en valeur, un dictionnaire prend n'importe quel type de données: listes, nombres, d'autres dictionnaires... On peut donc représenter des objects très complexes sous la forme de dictionnaires
    - Il s'écrit ainsi: `{"clé": "valeur"}`.
    - *exemple: `{"rouge": ["fraise", "tomate"], "vert": "poire"}`*

On va donc structurer notre corpus pour qu'il ait cette apparence:

```python
[
    {
        # première entrée du corpus
        "auteur": "Volaire",             # nom de l'auteur.ice. type `str`
        "date_creation": 1752,           # année de création. type `int`
        "date_vente": 1901,              # année de la vente. type `float`
        "prix": 100.1,                   # prix de vente en monnaie courante. type `float`
        "prix_constant": 105.1,          # prix de vente en francs constants 1900. type `str`
        "monnaie": "FRF",                # monnaie dans laquelle le prix courant est exprimé. type `str`
        "description": "Belle lettre"    # description de l'item en vente. type `str`
    },
    {
        # deuxième entrée du corpus
        ...
    },
    ...
]
```

Pour arriver à ce résultat, on va écrire une fonction qui prend en argument un `corpus` et retourne un `corpus_structure`. L'extraction d'information se base sur des méthodes de filtrage et de détection de motif dans du texte, et surtout sur des **expressions régulières** (regex). 

### Petit topo sur les expressions régulières

Les [regex](https://fr.wikipedia.org/wiki/Expression_r%C3%A9guli%C3%A8re) sont une méthode très répandue en informatique qui définit une syntaxe très puissante, mais un peu indigeste, pour la détection de motifs. 
- L'idée est de ne pas s'intéresser au contenu d'un texte, mais à **sa structure**: par exemple, `02/05/1972` et `10/12/1999` correspondent au motif: `[2 chiffres]/[2chiffres]/[4 chiffres]`. Il est alors possible de cibler ces deux dates par le même motif.
- À partir de là, les regex **regroupent les caractères en classes**: les lettres appartiennent à une classe, les chiffres à une autre... Les regex définissent des manières de représenter tous les membres d'une classe: `\d`, par exemple, représente "n'importe quel chiffre".
- Les regex définissent aussi: 
    - des **quantificateurs**: ils permettent de quantifier le nombre de fois que l'on cherche un caractère (`x?`: 0 ou 1 fois `x`, `x+`: 1 ou plusieurs fois `x`, `x*`: de 0 à *n* fois `x`...)
    - des **opérateurs d'aleternance** ("caractère A ou B" correspond à: `[xy]` ou `(x|y)`)
    - des manières de **définir des motifs négativement**: avec la syntaxe `[^x]`, où `x` est le caractère ou motif que l'on veut exclure
        - `[^\d]` permet de cibler tout ce qui n'est pas un chiffre.
    - un **système hiérarchique de groupes/sous-groupes**. Il est possible de découper une regex en plus petits motifs de façon hiérarchiques. Un sous-motif, ou *sous-groupe* est une partie d'une regex qui s'écrit entre parenthèses `()`. 
        - Les groupes sont numérotés dans une regex. Le premier *groupe* correspond à l'intégralité de la regex. Le deuxième au 1er élément entre parenthèses, et ainsi de suite.
        - Par exemple, dans `Prix: (\d+) euros`, `(\d+)` est un sous-motif.
- Enfin, elles permettent de **capturer et de réutiliser** des groupes ou des sous groupes: on peut récupérer une partie seulement d'une chaîne de caractère ciblée. 
    - Si on reprend l'exemple du dessus `Prix: (\d+) euros`, définir un sous-groupe permet de détecter une phrase entière mais de n'extraire que la valeur numéraire du prix: `\d+`. Le reste de la phrase donne alors le contexte du nombre recherché: on peut *cibler un élément en fonction de son contexte*.
    - Les groupes sont numérotés en fonction de leur ordre d'apparition: le groupe `0` est le motif entier, le groupe `1` est le premier sous-groupe en partant de la gauche...
   
La meilleure manière d'apprendre à utiliser des regex est de chercher *regex cheat sheet* sur internet et ensuite de faire des exercices, il y en a plein de ligne.

In [ ]:
def structure(corpus):
    """
    fonction pour représenter un corpus en texte brut en une liste de dictionnaires.
    """
    corpus_structure = []  # variable de sortie
    
    # 1) on transforme le corpus en texte brut en une liste d'entrées: 
    #
    # - on utilise la fonction `.split()` qui scinde un texte
    #   en une liste en fonction d'un séparateur (ici `\n\n\n`) 
    # - chaque entrée est séparée par un double saut de lignes 
    #   (= un triple retour à la ligne) => notre séparateur est `\n\n\n`
    corpus = corpus.split("\n\n\n")
    
    for entree in corpus:
        # variables dans lesquelles on stockera le contenu de nos entrées
        auteur = ""
        date_creation = ""
        date_vente = ""
        prix = ""
        monnaie = ""
        prix_constant = ""
        description = ""
        
        # 2) on ne traite pas les entrées vides et on supprime les autres entrées
        #
        # EN MOINS BREF: on ne traite que les éléments non-vides dans notre corpus 
        # (un élément vide = une entrée de liste qui ne contient rien, ou que des 
        # sauts de ligne et des espaces).
        # pour ce faire, on passe les entrées qui correspondent à la regex
        # `^(\n|\s)*$`:
        # - `^` = début de la chaîne de caractère
        # - `$` = fin de la chaîne de caractère sur laquelle on fait la recherche
        # - `\n` = retour à la ligne
        # - `\s` = un espace horizontal
        # - `(\n|\s)` signifique "soit `\n` soit `\s`
        # - `*` est un quantificateur qui indique que l'on cible 0 à n fois le groupe 
        #   précédent. ici, le groupe précédent est `(\n|\s)`
        # => la regex cible toutes les entrées qui sont soit vides, soit qui contiennent 
        #   seulement un ou plusieurs espaces et sauts de lignes
        if not re.search("^(\n|\s)*$", entree, flags=re.MULTILINE):
            
            # 3) on extrait l'auteur.ice
            #
            # l'auteur.ice est la première ligne d'une entrée => il suffit de la sélectionner
            auteur = entree.split("\n")[0]  # on sépare notre entrée en liste de lignes et on sélectionne la 1ere ligne 
            
            # 4) on extrait la description.
            # 
            # pareil, la description est toujours la 4e ligne d'une entrée.
            description = entree.split("\n")[3]
            
            
            # 5) on extrait la date de creation
            #
            # l'expression régulière pour cibler une date est `\d{4}`.
            # - `\d` = un chiffres
            # - `{n}` = un quantificateur qui signifie "cibler le groupe précédent n fois"
            # => on cible 4 chiffres.
            # pour extraire la bonne date, on rajoute le contexte: `"Écrit en \d{4}"`.
            #
            # entourer `\d{4}` de parenthèses en fait un sous-groupe: on pourra récupérer la
            # date seulement en récupérant. `re.search()` retourne les éléments du texte qui
            # correspondent au motif dans une liste: 
            # - l'élément à l'index 1 (`date_match[0]`) correspond au texte ciblé par
            #   l'intégralité de la regex: "Écrit en 1801", par exemple.
            # - l'élément à l'index 2 (`date_match[1]`) correspond au premier sous-groupe: la date.
            match = re.search("Écrit en (\d{4})", entree)
            if match:                     # si un élément dans l'entrée correspond à la regex
                creation = int(match[1])  # `int()` permet de transformer du texte en un nombre
            
            # 6) on extrait la date de vente
            #
            # le processus est exactement le même: on définit une regex
            # qui donne du contexte et on extrait un sous-groupe qui forme la
            # date en elle-même. on change le type (retype) de cette date: du texte,
            # on passe au nombre.
            match = re.search("Vendu en (\d{4})", entree)
            if match:
                date_vente = int(match[1])
    
            # 7) on extrait le prix en monnaie courante
            #
            # le principe est le même, mais on complexifie la regex:
            # avec une seule regex, on veut cibler 
            # - le prix (un nombre entier ou décimal)
            # - la monnaie (une ou plusieurs lettres majuscules)
            #
            # d'abord, le prix:
            # - un nombre entier, c'est un ou plusieurs chiffres: `\d+`
            # - un nombre décimal, c'est :
            #   - un ou plusieurs chiffres: `\d+`
            #   -  suivi d'un point (`\.`: précédé d'un backslash parce que le `.`
            #      est un caractère spécial en regex qui veut dire: n'importe 
            #      quel caractère)
            #   - suivi d'un ou plusieurs chiffres.
            # - on veut cibler soit un nombre entier, soit un nombre décimal. Pour ce
            #   faire, on utilise des quantificateurs pour dire: 
            #   "Je veux la partie entière d'un prix et la partie décimale si elle existe".
            #   => le premier `\d+` est obligatoire, le point suivi de la décimale
            #   sont optionnels. on arrive donc à:
            #   `\d+\d.?\d+`: "1+ chiffres suivi de 0 ou 1 point suivi de 0+ chiffres"
            #
            # ensuite la monnaie: c'est simple, c'est une capitale qui se trouve juste après
            # le prix:
            # - `[]` désigne une alternance: on cible n'importe quel caractère à l'intérieur
            #   des crochets
            # - le tiret `-` permet de cibler tous les caractères entre celui de gauche
            #   et celui de droite. Ici, on cible donc tous les caractères entre A et Z.
            # - le quantificateur `+` permet d'indiquer 1 ou plusieurs => on cible 1 ou 
            #   plusieurs lettres majuscules.
            #
            # enfin, l'organisation globale: on cherche: 
            # "Prix: [nombre entier ou décimal] [monnaie]".
            # pour cibler le prix et la monnaie, on met chacun des motifs expliqués
            # ci-dessus entre `()`. On  a donc une regex avec 3 groupes (1 motif + 2 sous-groupes)
            match = re.search("Prix: (\d+\.?\d*) ([A-Z]+)", entree)
            if match:
                prix = float(match[1])  # le 1er groupe capturé = le prix. On en fait un nombre décimal
                monnaie = match[2]      # le 2e groupe capturé = la monnaie
            
            # 8) rebelote pour le prix en francs constants.
            # 
            # on cible la encore un nombre entier ou décimal: `\d+\.?\d*`
            # seul le contexte change: on veut que ce nombre soit précédé par:
            # "en francs constants 1900: ".
            match = re.search("en francs constants 1900: (\d+\.?\d*)", entree)
            if match:
                prix_constant = float(match[1])  # on extrait le nombre et en fait un nombre décimal
            
            # 9) pour finir
            # 
            # on a ciblé tous nos éléments! pour finir, on construit notre entrée
            # et on l'ajoute à `corpus_structure`
            entree = {
                "auteur": auteur,
                "date_creation": date_creation,
                "date_vente": date_vente,
                "prix": prix,
                "prix_constant": prix_constant,
                "monnaie": monnaie,
                "description": description
            }
            corpus_structure.append(entree)

    return corpus_structure  # on retourne la variable créée pour l'utiliser en dehors de la fonction


corpus_idees = structure(corpus_idees)
corpus_roman = structure(corpus_roman)
corpus_poeme = structure(corpus_poeme)
corpus_theatre = structure(corpus_theatre)

---

## Visualisation: faire sens du corpus

À ce stade, on a probablement fini le plus difficile et le plus intéressant du cours: à partir d'un texte brut, on est arrivé.es à un résultat structuré. Il est maintenant possible de dire à un ordinateur de cibler telle ou telle partie du corpus, de faire des calculs... Par exemple:

In [ ]:
print(corpus_idees[:1])  # imprimer la dernière entrée de `corpus_idees`
print(corpus_poeme[5]["auteur"], "-", corpus_poeme[5]["description"])  # l'auteur.ice de la 6e entrée du `corpus_poeme` et sa description

Maintenant, la première question est peut-être: qu'est-ce que l'on cherche à comprendre de ce corpus? Avec deux questions connexes: 
- Qu'est-ce qu'il est possible de comprendre, relativement facilement, par une analyse computationnelle?
- Qu'est-ce qui peut être visualisé relativement facilement?

Le plus simple pour faire sens du corpus, c'est d'utiliser de méthodes quantitatives. Avec des calculs simples, on va essayer de comprendre comment nos quatre genres littéraires sont représentés dans des catalogues de vente dans le temps. Pour produire des résultats compréhensibles, les résultats de nos calculs seront représentés dans 2 graphiques:
- Le premier représente le nombre de manuscrits mis en vente pour chaque genre par an
- Le deuxième représente l'évolution du prix médian d'un manuscrit, par genre et par an

Pour les graphiques, on va utiliser la librairie python `Plotly` qui permet de produire des graphiques interactifs (sur le Web) ou statiques (des fichiers images). Le concept de base est très simple: **un graphique 2D est un objet produit par Plotly en lui donnant 2 listes de valeurs, une pour les abscisses (`x`) et une (ou plusieurs) pour les ordonnées (`y`)**.

### 1) Préparer les données

On va devoir **maniupler nos données pour produire**:
- un axe des abscisses `x` (une liste de toutes les années entre la plus ancienne vente et la vente la plus récente, pour les 4 corpus)
- des axes des ordonnées `y`. Comme on fait 2 graphiques, on devra produire un 2 listes pour nos ordonnées `y` par corpus:
    - pour le premier graphique, le nombre de manuscrits mis en vente par an pour ce corpus
    - pour le second, le prix médian pour un manuscrit par an, pour ce corpus.

Dans la manipulation des données, il y a **deux petites subtilités**: 
- **l'axe des `y` doit toujours être aligné à l'axe des `x`**: on produit une liste pour chaque axe, mais les valeurs d'une liste doivent être alignées. Dans l'exemple ci-dessous, il n'y a pas de prix pour l'année 1902. Si on met en `x` les années et en `y` les prix, on devra prendre en compte l'absence de valeurs pour 1902. 
    - Sinon, on aurait 2 listes `x=[1900,1901,1902,1903]` (4 valeurs) et `y=[3.0,3.1,3.5]` (3 valeurs). Il y aurait donc un décalage entre les années et les prix à partir de 1902.
    - La représentation correcte de nos données serait: `x=[1900,1901,1902,1903]` et `y=[3.0,3.1,0.0,3.5]` (la valeur `0.0`signifiant ici l'abscence de valeur).

    |           |1900|1901|1902|1903|
    |-----------|----|----|----|----|
    |Prix médian| 3.0| 3.1| n.a| 3.5|
    
- les ordonnées `y` de nos 4 corpus doivent pouvoir **être représentés sur le même axe des abscisses `x`**. Si `corpus_poeme` va de 1901 à 1915 et que `corpus_roman` va de 1870 à 1910, alors l'axe `x` et les axes `y` des 2 corpus devront être alignés sur la même tranche de dates: `1870-1915`.

On commence par **définir une fonction pour produire des données pour les `x` et `y` de chaque corpus**:
- `x` sera une liste de toutes les années entre l'année de la vente la plus ancienne et l'année de la vente la plus récente
- `y_prix` sera un dictionnaire qui associe à chaque année le prix médian pour un item, en francs constants
- `y_count` sera un dictionnaire qui associe à chaque année le nombre de manuscrits mis en vente

Pour les axes des abscisses, on utilise pour le moment des dictionnaires pour garder le lien entre les dates et les valeurs. On transformera plus tard ces axes en listes.

In [ ]:
def axes(corpus):
    """
    produire des données pour les x et y d'un corpus

    :param corpus: le corpus à traiter.
    :returns: - x : une liste de toutes les années entre l'année de la vente 
                    la plus ancienne et l'année de la vente la plus récente
              - y_prix : un dictionnaire qui associe à chaque année le prix 
                         médian pour un item, en francs constants
              - y_count : sera un dictionnaire qui associe à chaque année le 
                          nombre de manuscrits mis en vente
    """
    # variables de sortie
    x = []
    y_prix = {}
    y_count = {}
    
    # ci-dessous, on utilise une syntaxe spéciale qui permet de faire une boucle `for`
    # sans retour à la ligne, pour créer une ligne plus rapidement
    annees = [ entree["date_vente"] for entree in corpus ]  # liste d'années de vente
    x = [ annee for annee in range(min(annees), max(annees) + 1) ]    # liste d'années entre la date de vente la plus ancienne et la plus récente
    
    # ensuite, on ajoute les années clés de nos dictionnaires `y_...`. ces clés
    # sont les années à l'intérieur de `x`: on garde ainsi le lien entre abscisses 
    # et ordonnées
    y_prix = { annee: [] for annee in x }  # on associe à chaque année une liste vide: on y ajoutera tous les prix de vente
    y_count = { annee: 0 for annee in x }  # on associe à chaque année un 0. on comptera toutes les ventes pour cette année
    
    # on ajoute ensuite des valeurs à nos dictionnaires `y_...`
    for entree in corpus:
        date = entree["date_vente"]
        
        # on ajoute le prix constant à `y_prix`, à la bonne date
        if entree["prix_constant"] != "":
            y_prix[date].append(entree["prix_constant"])
        
        # on augmente le compteur de ventes par an de 1 dans `y_count`
        y_count[date] += 1
        
    # enfin, on calcule le prix de vente médian pour une année dans `y_prix`:
    for annee, prix in y_prix.items():
        if len(prix) > 0:
            y_prix[annee] = median(prix)  # si il y a des prix pour cette année, remplacer la liste de prix par le prix médian
        else:
            y_prix[annee] = 0  # si il n'y a pas eu de vente pour cette année, le prix médian est de 0.
    
    return x, y_prix, y_count


# on définit nos axes des x et des y.
x_idees, y_prix_idees, y_count_idees = axes(corpus_idees)
x_poeme, y_prix_poeme, y_count_poeme = axes(corpus_poeme)
x_roman, y_prix_roman, y_count_roman = axes(corpus_roman)
x_theatre, y_prix_theatre, y_count_theatre = axes(corpus_theatre)

print(y_count_idees)
print(y_prix_idees)

On a maintenant des valeurs pour chaque corpus. Les années dans les différents corpus sont cependant différentes. Il faut donc retraiter les données pour pouvoir **représenter les données des 4 corpus sur la même tranche d'années et sur le même axe `x`**.

On commence par créer un unique axe des abscisses `x` à partir de nos 4 `x_...` créés au dessus. 

In [ ]:
# on crée une liste qui regroupe toutes les années
# de vente des 4 corpus
annees_total = []
for annee in x_idees:
    annees_total.append(annee)
for annee in x_poeme:
    annees_total.append(annee)
for annee in x_roman:
    annees_total.append(annee)
for annee in x_theatre:
    annees_total.append(annee)

# à ce stade, il peut y avoir des doublons et nos années
# ne sont pas ordonnées. on génère donc un axe des `x` avec
# toutes les années entre les dates extrêmes de `annees_total`,
# dans l'ordre croissant
x = [ annee for annee in range(min(annees_total), max(annees_total) + 1) ]
print(x)

On progresse ! On a maintenant un axe des abscisses unique. Il s'agit ensuite de transformer tous nos `y`: ce sont des dictionnaires, on doit en faire des listes alignées sur l'axe des `x`. 

On écrit donc une fonction:

In [ ]:
def align_y_to_x(x, y_dict):
    """
    transformer un dictionnaire `y_dict` associant une année à une valeur en une
    liste alignée sur l'axe des abscisses `x`
    """
    y_out = []  # variable de sortie
    
    for annee in x:
        if annee in y_dict.keys():
            # si il y a des données pour cette année dans y_dict, on les ajoute
            y_out.append(y_dict[annee])
        else:
            # si il n'y a pas de données, on ajoute un 0
            y_out.append(0)
    
    return y_out

# on crée tous nos axes des `y`. ce sont maintenant des listes de
# valeurs numéraires alignés sur `x`. 
y_prix_idees = align_y_to_x(x, y_prix_idees)
y_count_idees = align_y_to_x(x, y_count_idees)
    
y_prix_poeme = align_y_to_x(x, y_prix_poeme)
y_count_poeme = align_y_to_x(x, y_count_poeme)
    
y_prix_theatre = align_y_to_x(x, y_prix_theatre)
y_count_theatre = align_y_to_x(x, y_count_theatre)
    
y_prix_roman = align_y_to_x(x, y_prix_roman)
y_count_roman = align_y_to_x(x, y_count_roman)


# syntaxe un peu moche mais compacte: on affiche 
# le prix médian par an pour le corpus de littérature d'idées.
for a in x:
    print(f"{a}: {y_prix_idees[x.index(a)]}")

### 2) Visualiser les données

Nos données sont prêtes ! Il ne s'agit plus que de les utiliser pour produire des visualisations avec Plotly. L'utilisation de cette librairie n'est pas le point central de ce cours, donc on va passer vite dessus. Elle est très personnalisable et un peu difficile à prendre en main au début (mais on finit très vite par faire sa recette qui fonctionne et l'adapter pour faire des nouveaux graphiques).

#### Plotly en bref 

Pour rappel, on utilise un sous-module de Plotly appelé `graph_objects`, que l'on a importé en le renommant `go`. Quand il est écrit `go.QuelqueChose()`, il s'agit donc d'une fonction issue du module  `graph_objects` de la librairie `plotly`.

un graphique est un objet nommé `Figure`. Il est composé de 2 parties: les données (`data`) et la mise en page (`layout`).
- `data`: les données à afficher en ordonnée. 
    - Il y a une entrée dans la liste par courbe à afficher. 
    - Chaque entrée de la liste est elle-même un objet Plotly, qui indique comment représenter les données `y` produites ci-dessus.
    - par exemple, `go.Bar()` produit un graphique en barres, `go.Scatter()` un *scatter plot*.
- `layout`: la mise en page est elle-même un objet plotly: `go.Layout()`, auquel on fournit un dictionnaire de paramètres.

On définit des variables pour la mise en page qui seront utilisées par les 2 graphiques:

In [ ]:
colors = {
    "white": "#ffffff", "cream": "#fcf8f7", "blue": "#0000ef", 
    "burgundy": "#890c0c", "pink": "#ff94c9", "gold": "#da9902", 
    "lightgreen": "#8fc7b1", "darkgreen": "#00553e", "peach": "#ffad98"
}  # couleurs HTML
layout = {
    "paper_bgcolor": colors["white"],
    "plot_bgcolor": colors["cream"],
    "margin": {"l": 50, "r": 50, "t": 50, "b": 50},
    "showlegend": True,
    "xaxis": {"anchor": "x", "title": {"text": r"Année"}},
    "barmode": "stack"
}  # mise en page de base

#### On commence par faire le graphique sur le prix médian d'un manuscrit, par corpus et par an

Comme on traite 4 axes à la fois, on définit une variable `corpus` qui stocke toutes les informations nécessaires pour créer une visualisation sur nos 4 corpus sous la forme d'une liste imbriquée. Ce n'est pas obligatoire, mais ça permet d'écrire moins de code: plutôt que de réécrire 4 fois le même code, il suffira de boucler sur chaque item de la liste pour créer un objet `go.Bar()` qui représente une courbe. La structure de notre variable `corpus` est la suivante:

```python
[
    [ "données 1", "titre 1", "couleur 1" ],
    [ "données 2", "titre 2", "couleur 2" ],
    ...
]
```

In [ ]:
corpus = [
    [ y_prix_idees, "Corpus idées", colors["peach"] ]
    , [ y_prix_theatre, "Corpus théâtre", colors["lightgreen"] ]
    , [ y_prix_poeme, "Corpus poèmes", colors["pink"] ]
    , [ y_prix_roman, "Corpus roman", colors["gold"] ]
]
ordonnees = []

# pour chaque entrée de `corpus`, on crée un objet `go.Bar()` qui
# représente une courbe et on l'ajoute à notre variable `ordonnees`
for axe in corpus:
    # on crée l'objet
    bar = go.Bar(
        x=x,                       # valeurs en abscisse
        y=axe[0],                  # valeurs en ordonnées
        marker={"color": axe[2]},  # couleur
        name=axe[1]                # titre
    )
    # on l'ajoute à `ordonnees`
    ordonnees.append(bar)

# enfin, on met à jour notre `layout` avec des informations spécifiques
# à ce graphique et on crée l'objet `go.Layout()`
layout["yaxis"] = {"anchor": "x", "title": {"text": "Prix médian d'un manuscrit"}}
layout["title"] = "Prix médian d'un manuscrit, en fonction du genre littéraire pour lequel l'auteur.ice est connu.e, par an"
layout = go.Layout(layout)

# pour finir, on crée notre figure: la structure de l'objet `go.Figure()` est clairement visible ici!
fig_prix = go.Figure(
    data=ordonnees,
    layout=layout
)
fig_prix.show()

#### On fait ensuite un graphique sur le nombre de manuscrits en vente, par corpus et par an

Pour cette visualisation, on va faire une représentation en *scatterplot*, et non en barres. On utilise donc un `go.Scatter()` au lieu de `go.Plot()`. Sinon, la logique est exactement la même que ci-dessus!

In [ ]:
corpus = [
    [ y_count_idees, "Corpus idées", colors["peach"] ]
    , [ y_count_theatre, "Corpus théâtre", colors["lightgreen"] ]
    , [ y_count_poeme, "Corpus poèmes", colors["pink"] ]
    , [ y_count_roman, "Corpus roman", colors["gold"] ]
]
ordonnees = []

# on crée nos ordonnées
for axe in corpus:
    # le processus est exactement le même qu'au dessus: pour chaque `axe`
    # de notre liste, on crée un objet `go` plotly et on l'ajoute à `ordonnées`.
    # pour plus de variété, on utilise une autre forme graphique.
    scatter = go.Scatter(
        x=x,               # valeur en abscisse
        y=axe[0],          # valeurs en ordonnées
        fillcolor=axe[2],  # la couleur de remplissage
        stackgroup="one",  # indication qu'il faut empiler les axes
        orientation="v",   # empiler les axes à la verticale
        name=axe[1]        # le nom à donner à chaque axe
    ) 
    ordonnees.append(scatter)

# on met à jour la mise en page
layout["yaxis"] = {"anchor": "x", "title": {"text": "Nombre de manuscrits mis en vente"}}
layout["title"] = "Nombre de manuscrits mis en vente, en fonction du genre littéraire pour lequel l'auteur.ice est connu.e, par an"
layout = go.Layout(layout)

# on crée notre figure et on la montre
fig_count = go.Figure(
    data=ordonnees,
    layout=layout
)
fig_count.show()

# pour finir, on sauvegarde les fichiers en sortie avec la fonction `write_image`
# de Plotly. pour enregistrer une visualisation interactive, utiliser `write_html`.
fig_prix.write_image("prix_median_par_an.png")
fig_count.write_image("nombre_items_par_an.png")

## Pour conclure

Voilà, c'est fini! En travaillant depuis un texte brut, on a produit une structure, une analyse (basique) et une visualisation (basique) de celui-ci. Tout ce travail a été fait avec des outils relativement basiques et disponibles de base en Python (à l'exception de Plotly). En fait, tout le travail de structuration du texte, d'extraction et d'organisation des informations est fait entièrement à l'aide de "pur python" (on a pas utilisé de librairies externes), à l'exception de quelques petites regex. 

À partir des deux graphiques, on peut essayer de comprendre (un peu) comment sont perçus quelques genres littéraires du XVIIIe siècle au XIXe siècle. Il aurait été intéressant d'exprimer la répartition des genres et des prix en pourcentages du corpus complet plutôt en valeurs absolues, mais c'est une autre paire de manches ! 

Cela aurait aussi été utile de travailler sur des corpus plus volumineux, ce qui demanderait d'avoir plus d'informations sur les auteur.ice.s de manuscrits présent.es dans le corpus de Katabase. Un alignement avec Wikidata a été réalisé par le projet, pour constituer une base de données [disponible ici](https://github.com/katabase/3_WikidataEnrichment) sur les auteur.ice.s de Katabase. Un croisement entre cette base (en `json`) et les corpus de Katabase permettrait d'obtenir des résultats vraiment plus intéressants sur la représentation des genres littéraires du XVIIIe siècle dans le marché du manuscrit du XIXe siècle.

Bien sûr, si le traitement a été aussi facile, c'est qu'on a travaillé avec un corpus particulièrement structuré. Et pour cause, c'est moi qui ait produit ce corpus à partir de l'API mise en ligne par Katabase.

### Pour apprendre à créer des corpus de recherche  à partir de sources de données en ligne, c'est dans le notebook suivant : )

## [Notebook suivant](https://colab.research.google.com/drive/15veTQ_VsmwAwG9RAijibkO9COg3QuDq-?usp=sharing)